This is in order to help Flooded People community organisers identify who they would like to speak to first, I'm identifying where there are geographical overlaps. 

In [78]:
import geopandas as gpd
import pandas as pd

In [79]:
efz_path = "/Users/eclough_98/flooded-people/fp-flood-nvfi/data/processing/filtered-fz/filtered-fz.shp"
england_fz = gpd.read_file(efz_path)

nfvie_path = "/Users/eclough_98/flooded-people/fp-flood-nvfi/data/processing/nfvi-eng/nfvi-eng.shp"
eng_nfvi = gpd.read_file(nfvie_path)

sfz_path = "/Users/eclough_98/flooded-people/fp-flood-nvfi/data/processing/scotland-fz/scotland-fz.shp"
scotland_fz = gpd.read_file(sfz_path)

nfvis_path = "/Users/eclough_98/flooded-people/fp-flood-nvfi/data/processing/nfvi-scot/nfvi-scot.shp"
scot_nfvi = gpd.read_file(nfvis_path)

scotland_fz = scotland_fz.to_crs(27700)
england_fz = england_fz.to_crs(epsg=27700)
scot_nfvi = scot_nfvi.to_crs(epsg=27700)
eng_nfvi = eng_nfvi.to_crs(epsg=27700)

In [62]:
print(england_fz.head())
print(eng_nfvi.head())

  flood_zone                                           geometry
0        FZ2  POLYGON ((-1.77893 52.48745, -1.77893 52.48743...
1        FZ2  POLYGON ((-1.9165 52.46625, -1.9165 52.46627, ...
2        FZ2  MULTIPOLYGON (((-1.93392 52.45184, -1.93392 52...
3        FZ2  POLYGON ((-1.92878 52.48188, -1.92872 52.48188...
4        FZ2  POLYGON ((-1.9234 52.40763, -1.9234 52.40765, ...
   nfvi-uk                           Name  \
0    1.342  Relatively High Vulnerability   
1    0.968  Relatively High Vulnerability   
2    0.797  Relatively High Vulnerability   
3    0.597  Relatively High Vulnerability   
4    0.581  Relatively High Vulnerability   

                                            geometry  
0  POLYGON ((-0.05428 51.53244, -0.05428 51.53201...  
1  POLYGON ((-0.05775 51.53223, -0.05744 51.53134...  
2  POLYGON ((-0.04485 51.53204, -0.04451 51.53185...  
3  POLYGON ((-0.05931 51.52989, -0.059 51.5288, -...  
4  POLYGON ((-0.06563 51.53149, -0.06557 51.53133...  


We expect that community organisers will want to reach:
Stage 1: Extremely and Very Flood Vulnerable People in High Risk Floodzones
Stage 2: Relatively High Flood Vulnerable People in High Risk Floodzones *and* Extremely and Very Flood Vulnerable People in Medium Risk Floodzones
Stage 3: Relatively High Flood Vulnerable People in Medium Risk Floodzones

"Acute Vulnerability", "Extremely High Vulnerability", "Relatively High Vulnerability"

In [80]:
eng_high_flood = england_fz[england_fz["flood_zone"].isin(["FZ3"])]
eng_medium_flood = england_fz[england_fz["flood_zone"].isin(["FZ2"])]

high_vuln_eng = eng_nfvi[eng_nfvi["Name"].isin(["Acute Vulnerability", "Extremely High Vulnerability"])]
medium_vuln_eng = eng_nfvi[eng_nfvi["Name"]=="Relatively High Vulnerability"]

eng_high_flood.geometry = eng_high_flood.geometry.buffer(0)
eng_medium_flood.geometry = eng_medium_flood.geometry.buffer(0)
high_vuln_eng.geometry = high_vuln_eng.geometry.buffer(0)
medium_vuln_eng.geometry = medium_vuln_eng.geometry.buffer(0)

cat_one_engl = high_vuln_eng.clip(eng_high_flood)
cat_one_engl["name"] = "Level 1"
cat_two_engl1 = medium_vuln_eng.clip(eng_high_flood)
cat_two_engl2 = high_vuln_eng.clip(eng_medium_flood)
cat_two_engl = pd.concat([cat_two_engl1, cat_two_engl2], ignore_index=True) 
cat_two_engl["name"] = "Level 2" 
cat_three_engl = medium_vuln_eng.clip(eng_medium_flood)
cat_three_engl["name"] = "Level 3"

cat_england = pd.concat([cat_one_engl, cat_two_engl, cat_three_engl], ignore_index=True) 

cat_england = cat_england.to_crs(27700)
cat_england["geometry"] = cat_england.simplify(tolerance=1.5, preserve_topology=True)
cat_england.geometry = cat_england.buffer(0)
cat_england = cat_england.to_crs(4326)

cat_england = cat_england[["name", "geometry"]]

ec_path = "/Users/eclough_98/flooded-people/fp-flood-nvfi/data/clean/england/eng_cat_england.kml"
cat_england.to_file(ec_path, driver="KML", index=False)


/Users/eclough_98/flooded-people/fp-flood-nvfi/.venv/lib/python3.13/site-packages/geopandas/geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [81]:
scot_high_flood = scotland_fz[scotland_fz["PROB"].isin(['High Risk of Flooding Rivers & Seas'])]
scot_medium_flood = scotland_fz[scotland_fz["PROB"].isin(['Medium Risk of Flooding Rivers & Seas'])]

high_vuln_scot = scot_nfvi[scot_nfvi["Name"].isin(["Acute Vulnerability", "Extremely High Vulnerability"])]
medium_vuln_scot = scot_nfvi[scot_nfvi["Name"]=="Relatively High Vulnerability"]

scot_high_flood.geometry = scot_high_flood.geometry.buffer(0)
scot_medium_flood.geometry = scot_medium_flood.geometry.buffer(0)
high_vuln_scot.geometry = high_vuln_scot.geometry.buffer(0)
medium_vuln_scot.geometry = medium_vuln_scot.geometry.buffer(0)

cat_one_scotl = high_vuln_scot.clip(scot_high_flood)
cat_one_scotl["name"] = "Level 1"
cat_two_scotl1 = medium_vuln_scot.clip(scot_high_flood)
cat_two_scotl2 = high_vuln_scot.clip(scot_medium_flood)
cat_two_scotl = pd.concat([cat_two_scotl1, cat_two_scotl2], ignore_index=True) 
cat_two_scotl["name"] = "Level 2" 
cat_three_scotl = medium_vuln_scot.clip(scot_medium_flood)
cat_three_scotl["name"] = "Level 3"

cat_scotland = pd.concat([cat_one_scotl, cat_two_scotl, cat_three_scotl], ignore_index=True) 

cat_scotland = cat_scotland.to_crs(27700)
cat_scotland["geometry"] = cat_scotland.simplify(tolerance=.5, preserve_topology=True)
cat_scotland.geometry = cat_scotland.buffer(0)
cat_scotland = cat_scotland.to_crs(4326)

cat_scotland = cat_scotland[["name", "geometry"]]

sc_path = "/Users/eclough_98/flooded-people/fp-flood-nvfi/data/clean/scotland/scot_cat_scotland.kml"
cat_scotland.to_file(sc_path, driver="KML", index=False)



/Users/eclough_98/flooded-people/fp-flood-nvfi/.venv/lib/python3.13/site-packages/geopandas/geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
